In [2]:
import featuretools as ft

data = ft.demo.load_mock_customer()
transactions_df = data["transactions"].merge(data["sessions"]).merge(data["customers"])

transactions_df.sample(10)



,transaction_id,session_id,transaction_time,product_id,amount,customer_id,device,session_start,zip_code,join_date,birthday
264,296,20,2014-01-01 04:46:00,5,53.22,5,desktop,2014-01-01 04:46:00,60091,2010-07-17 05:27:50,1984-07-28
19,74,2,2014-01-01 00:20:35,1,106.99,5,mobile,2014-01-01 00:17:20,60091,2010-07-17 05:27:50,1984-07-28
314,141,23,2014-01-01 05:40:10,5,128.26,3,desktop,2014-01-01 05:32:35,13244,2011-08-13 15:42:34,2003-11-21
290,236,21,2014-01-01 05:14:10,5,57.09,4,desktop,2014-01-01 05:02:15,60091,2011-04-08 20:08:14,2006-08-15
379,292,28,2014-01-01 06:50:35,1,133.71,5,mobile,2014-01-01 06:50:35,60091,2010-07-17 05:27:50,1984-07-28
335,482,25,2014-01-01 06:02:55,1,26.30,3,desktop,2014-01-01 05:59:40,13244,2011-08-13 15:42:34,2003-11-21
293,452,21,2014-01-01 05:17:25,5,69.62,4,desktop,2014-01-01 05:02:15,60091,2011-04-08 20:08:14,2006-08-15
271,169,20,2014-01-01 04:53:35,3,78.87,5,desktop,2014-01-01 04:46:00,60091,2010-07-17 05:27:50,1984-07-28
404,476,29,2014-01-01 07:17:40,4,11.62,1,mobile,2014-01-01 07:10:05,60091,2011-04-17 10:48:33,1994-07-18
179,72,12,2014-01-01 03:13:55,2,143.96,4,desktop,2014-01-01 03:04:10,60091,2011-04-08 20:08:14,2006-08-15


In [3]:
products_df = data["products"]
products_df

,product_id,brand
0,1,B
1,2,B
2,3,B
3,4,B
4,5,A


In [4]:
es = ft.EntitySet(id="customer_data")

from woodwork.logical_types import Categorical, PostalCode

es = es.add_dataframe(
    dataframe_name="transactions",
    dataframe=transactions_df,
    index="transaction_id",
    time_index="transaction_time",
    logical_types={
        "product_id": Categorical,
        "zip_code": PostalCode,
    },
)

es

/mnt/c/Users/james/projects/auto-feature-engineering/.venv/lib/python3.10/site-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/mnt/c/Users/james/projects/auto-feature-engineering/.venv/lib/python3.10/site-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(


Entityset: customer_data
  DataFrames:
    transactions [Rows: 500, Columns: 11]
  Relationships:
    No relationships

In [5]:
es["transactions"].ww.schema

,Logical Type,Semantic Tag(s)
Column,,
transaction_id,Integer,['index']
session_id,Integer,['numeric']
transaction_time,Datetime,['time_index']
product_id,Categorical,['category']
amount,Double,['numeric']
customer_id,Integer,['numeric']
device,Categorical,['category']
session_start,Datetime,[]
zip_code,PostalCode,['category']


In [6]:
es = es.add_dataframe(
    dataframe_name="products", dataframe=products_df, index="product_id"
)

es

/mnt/c/Users/james/projects/auto-feature-engineering/.venv/lib/python3.10/site-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/mnt/c/Users/james/projects/auto-feature-engineering/.venv/lib/python3.10/site-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(


Entityset: customer_data
  DataFrames:
    transactions [Rows: 500, Columns: 11]
    products [Rows: 5, Columns: 2]
  Relationships:
    No relationships

In [7]:
es = es.add_relationship("products", "product_id", "transactions", "product_id")
es

Entityset: customer_data
  DataFrames:
    transactions [Rows: 500, Columns: 11]
    products [Rows: 5, Columns: 2]
  Relationships:
    transactions.product_id -> products.product_id

In [8]:
es = es.normalize_dataframe(
    base_dataframe_name="transactions",
    new_dataframe_name="sessions",
    index="session_id",
    make_time_index="session_start",
    additional_columns=[
        "device",
        "customer_id",
        "zip_code",
        "session_start",
        "join_date",
    ],
)
es

Entityset: customer_data
  DataFrames:
    transactions [Rows: 500, Columns: 6]
    products [Rows: 5, Columns: 2]
    sessions [Rows: 35, Columns: 6]
  Relationships:
    transactions.product_id -> products.product_id
    transactions.session_id -> sessions.session_id

In [9]:
es["transactions"].ww.schema

,Logical Type,Semantic Tag(s)
Column,,
transaction_id,Integer,['index']
session_id,Integer,"['foreign_key', 'numeric']"
transaction_time,Datetime,['time_index']
product_id,Categorical,"['category', 'foreign_key']"
amount,Double,['numeric']
birthday,Datetime,[]


In [10]:
es = es.normalize_dataframe(
    base_dataframe_name="sessions",
    new_dataframe_name="customers",
    index="customer_id",
    make_time_index="join_date",
    additional_columns=["zip_code", "join_date"],
)

es



Entityset: customer_data
  DataFrames:
    transactions [Rows: 500, Columns: 6]
    products [Rows: 5, Columns: 2]
    sessions [Rows: 35, Columns: 4]
    customers [Rows: 5, Columns: 3]
  Relationships:
    transactions.product_id -> products.product_id
    transactions.session_id -> sessions.session_id
    sessions.customer_id -> customers.customer_id

In [11]:
feature_matrix, feature_defs = ft.dfs(entityset=es, target_dataframe_name="products")

feature_matrix

/mnt/c/Users/james/projects/auto-feature-engineering/.venv/lib/python3.10/site-packages/featuretools/computational_backends/feature_set_calculator.py:785: FutureWarning: The provided callable <function mean at 0x7fad7c63d240> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ).agg(to_agg)
/mnt/c/Users/james/projects/auto-feature-engineering/.venv/lib/python3.10/site-packages/featuretools/computational_backends/feature_set_calculator.py:785: FutureWarning: The provided callable <function std at 0x7fad7c63d360> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  ).agg(to_agg)
/mnt/c/Users/james/projects/auto-feature-engineering/.venv/lib/python3.10/site-packages/featuretools/computational_backends/feature_set_calculator.py:785: FutureWarning: The p

,COUNT(transactions),MAX(transactions.amount),MEAN(transactions.amount),MIN(transactions.amount),SKEW(transactions.amount),STD(transactions.amount),SUM(transactions.amount),MODE(transactions.DAY(birthday)),MODE(transactions.DAY(transaction_time)),MODE(transactions.MONTH(birthday)),...,MODE(transactions.sessions.device),NUM_UNIQUE(transactions.DAY(birthday)),NUM_UNIQUE(transactions.DAY(transaction_time)),NUM_UNIQUE(transactions.MONTH(birthday)),NUM_UNIQUE(transactions.MONTH(transaction_time)),NUM_UNIQUE(transactions.WEEKDAY(birthday)),NUM_UNIQUE(transactions.WEEKDAY(transaction_time)),NUM_UNIQUE(transactions.YEAR(birthday)),NUM_UNIQUE(transactions.YEAR(transaction_time)),NUM_UNIQUE(transactions.sessions.device)
product_id,,,,,,,,,,,,,,,,,,,,,
1,102,149.56,73.429314,6.84,0.125525,42.479989,7489.79,18,1,7,...,desktop,4,1,3,1,4,1,5,1,3
2,92,149.95,76.319891,5.73,0.151934,46.336308,7021.43,18,1,8,...,desktop,4,1,3,1,4,1,5,1,3
3,96,148.31,73.001250,5.89,0.223938,38.871405,7008.12,18,1,8,...,desktop,4,1,3,1,4,1,5,1,3
4,106,146.46,76.311038,5.81,-0.132077,42.492501,8088.97,18,1,7,...,desktop,4,1,3,1,4,1,5,1,3
5,104,149.02,76.264904,5.91,0.098248,42.131902,7931.55,18,1,7,...,mobile,4,1,3,1,4,1,5,1,3
